In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [51]:
import p8_util
datadir = './data'

filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, feature_shape = p8_util.load_dataset(filename_dataset,is_label_encoded=True)

print("\nDataset : {}".format((x_train.shape, x_test.shape, y_train.shape, y_test.shape, nClasses, feature_shape)))

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 672, 224) (47, 672, 224) (414, 3) (47, 3)

Dataset : ((414, 672, 224), (47, 672, 224), (414, 3), (47, 3), 3, (672, 224))


In [49]:
x_train.shape
x_batch = p8_util.batch_coloredimage_serial_reshape(x_train)
x_batch.shape

(414, 672, 224)

In [28]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = True

In [29]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (26846): Aucun processus de ce type
kill: (26852)

#### Increase of train dataset against test dataset.

In [30]:
import p8_util_config
import NNAdaNetBuilder

#-----------------------------------------------------------------
# Get the number of convolutional layers for CNN network
# This will fixe the equivalent parameter of AdaNet num layers.
#-----------------------------------------------------------------

FEATURES_KEY = 'images'
#my_feature_columns, loss_reduction, tf_head = p8_util.get_tf_head(FEATURES_KEY, feature_shape, nClasses, nn_type='DNN')
if p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'CNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['conv_layer_num']
elif p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'RNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['rnn_layer_num']
elif p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'DNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['dnn_layer_num']

print("\n Number of convolutional layers= {}".format(layer_num))
oNNAdaNetBuilder = NNAdaNetBuilder.NNAdaNetBuilder(p8_util_config.dict_adanet_config, num_layers=layer_num)
oNNAdaNetBuilder.feature_shape = feature_shape
oNNAdaNetBuilder.output_dir = OUTPUT_DIR
oNNAdaNetBuilder.show()

print("\nMax steps= {} / Number of EPOCH={}".format(p8_util_config.MAX_STEPS,p8_util_config.NUM_EPOCHS))


 Number of convolutional layers= 1

*** NNAdaNetBuilder : NN Type=DNN

*** get_tf_head() : feature shape= (672, 224)

*** get_tf_head() : feature columns= [_NumericColumn(key='images', shape=(672, 224), default_value=None, dtype=tf.float32, normalizer_fn=None)]

*** make_config() : output dir= ./tmp/baseline/DNN/DNN


Adanet outputdir     : ............................ ./tmp/baseline/DNN
Adanet output log    : ............................ ./tmp/baseline/DNN/DNN
NN type              : ............................ DNN
Features shape       : ............................ (672, 224)
Units in dense layer : ............................ 128
Number of layers     : ............................ 1
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 0.005
Weights initializer  : ............................ xavier
Batch normalization  :

In [31]:
p8_util_config.dict_adanet_config

{'adanet_feature_columns': None,
 'adanet_feature_shape': None,
 'adanet_tf_head': None,
 'adanet_lambda': 0.005,
 'adanet_is_learn_mixture_weights': True,
 'adanet_initial_num_layers': 0,
 'adanet_num_layers': None,
 'adanet_nn_candidate': 2,
 'adanet_output_dir': './tmp/adanet',
 'adanet_nn_layer_config': {'nn_type': 'DNN',
  'nn_dropout_rate': 0.0,
  'nn_batch_norm': True,
  'nn_dense_layer_num': 1,
  'nn_dense_unit_size': 128,
  'nn_logit_dimension': 3,
  'nn_optimizer': <tensorflow.python.training.rmsprop.RMSPropOptimizer at 0x7f533fafc8d0>,
  'nn_seed': 42,
  'nn_initializer_name': 'xavier',
  'nn_layer_config': {'dnn_layer_num': 1, 'dnn_hidden_units': 128}}}

In [32]:
import p8_util

batch_test = 20
input_fn_param={'num_epochs':p8_util_config.NUM_EPOCHS,\
                'batch_size':p8_util_config.BATCH_SIZE,\
                'feature_shape': oNNAdaNetBuilder.feature_shape,\
               }
train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        max_steps=p8_util_config.MAX_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train,input_fn_param),\
        steps=None,\
        throttle_secs=1)

In [33]:
params = {'net_builder': oNNAdaNetBuilder}
    
classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=oNNAdaNetBuilder.classifier_config)

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/DNN/DNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f53201c76a0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [34]:
import p8_util

start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.

*** input_fn() : feature_shape= (672, 224) / Label shape= [3]

*** preprocess_image() : features= {'images': <tf.Tensor 'Reshape:0' shape=(224, 224, 3) dtype=float32>} / label shape= (3,)
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 3)
**** *** _build_dnn_subnetwork : Layers= 1


*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into .

In [35]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))
print("------------------------------------------------\n")



------------------------------------------------
Time (sec) 64.64970374107361
DNN_EVAL_ACCURACY: 0.35990339517593384
LOSS: 1.0828536748886108
GLOBAL_STEP: 50
------------------------------------------------



In [36]:
oNNAdaNetBuilder.output_dir_log

'./tmp/baseline/DNN/DNN'

In [37]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(oNNAdaNetBuilder.output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')